# Electron Spin Resonance

Now using similar functions to the updated pauli notebook.

In [1]:
%matplotlib inline
import local_broom

Added 'quench/' and 'quench/libraries' for easy importing.


In [2]:
# Automagically reload modules when they are edited
%load_ext autoreload
%autoreload 2

In [17]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from qcodes import Instrument, Station
from monty import Monty
import MDAC
from may.dots import getvoltages
from may.custom_devices import connect_to_gb, newSiDot
from liveplot import LivePlot
from fridge import Fridge
import esr
from shfqc import SHFQC
from monty import Monty
import time
import logging

In [38]:
channels = ["ST", "P1", "P2", "J", "MW"]
channel_addr = [0, 1, 2, 3, 5]

shfqc = SHFQC(channels, channel_addr, "measure")
shfqc.connect()

Connecting to device DEV12158
Creating new command tables


In [5]:
try:
    mdac = Instrument.find_instrument("mdac")
    mdac.close()
except KeyError:
    print('Attempting to remove instrument with name mdac. Does not exist')

mdac = MDAC.MDAC('mdac', 'ASRL11::INSTR')

try:
    lockin = Instrument.find_instrument("sr860_top")
    lockin.close()
except KeyError:
    print("Cannot remove instrument with name sr860_top. Does not exist")

scfg = Station(config_file='/Users/LD2007/Documents/Si_CMOS_james/measurements/system.yaml')

lockin = scfg.load_instrument('sr860_top')
# psg = scfg.load_instrument('PSG')

gb_control_si = connect_to_gb(mdac)  # Create our custom MDAC mappings
si = newSiDot(mdac)
fridge = Fridge("BlueFors_LD")

Attempting to remove instrument with name mdac. Does not exist
Connected to: QNL Sydney MDAC (serial:0x32353532353551040030003E, firmware:1.8) in 4.22s
Cannot remove instrument with name sr860_top. Does not exist
Connected to: Stanford_Research_Systems SR860 (serial:004279, firmware:V1.51) in 0.12s
Loading GB driver and connecting
Manually Open Breakout_box_top_ch15_LCB
Manually Open Breakout_box_bot_ch09_RCB
Manually Open Breakout_box_bot_ch17_RG
Manually Open Breakout_box_top_ch17_ResB
Manually Open Breakout_box_top_ch19_P3
Manually Open Breakout_box_top_ch13_P2
Manually Open Breakout_box_top_ch04_P1
Manually Open Breakout_box_top_ch02_SETB
Manually Open Breakout_box_bot_ch04_SRB
Manually Open Breakout_box_bot_ch06_SLB
Manually Open Breakout_box_top_ch10_ST
Manually Open Breakout_box_top_ch25_bias
Using base URL https://qphys1114.research.ext.sydney.edu.au/therm_flask/BlueFors_LD


In [6]:
#from qcodes.instrument_drivers.agilent.E8267C import E8267

#psg = E8267('psg', 'TCPIP0::192.168.0.102::inst0::INSTR')

# need to configure PSG manually

In [53]:
# Disconnect from all devices
disconnect = True
if disconnect:
    shfqc.disconnect()
    try:
        mdac = Instrument.find_instrument("mdac")
        mdac.close()
    except KeyError:
        print('Attempting to remove instrument with name mdac. Does not exist')
    try:
        lockin = Instrument.find_instrument("sr860_top")
        lockin.close()
    except KeyError:
        print("Cannot remove instrument with name sr860_top. Does not exist")

Disconnecting from device DEV12158


In [8]:
# setup monty
experiment = {
    "desc": "Run ESR frequency sweep to determine qubit frequencies"
}

monty = Monty("rf.esr", experiment)

Loading existing experiment (ignoring given experiment parameters)
Note that no experimental data has been loaded.
Next run will have id 18


In [49]:
params = {
    "amplitude_volts": {  # define the amplitudes that should be used. (maximum 340mV, give in Volts)
        "mixed_pulse": {  # voltage amplitude for mixed state preperation
            "P1": -0.2,  # make sure keys match drive lines above
            "P2": 0.2,
        },
        "readout_pulse": {
            "P1": -0.01,
            "P2": 0.01,
            "J": 0.0,
        },
        "ramp_rate": 13e-5,  # V/sample
    },
    "readout": {
        "freq": 406.6e6,  # (Hz) try 406.2e6
        "gain": 0.95,  # If we set this to 1, then output overloads
        "time": 100e-6,  # sec
    },
    "mw": {  # Microwave ESR settings
        "freqs": {  # (Hz)
            "start": 700e6,
            "stop": 800e6,
            # steps determined by averaging/num_freqs
        },
        "psgfreq": 13.9e9,
        "gain": 0.7,  # If we set this to 1, then output overloads
        "span": 1e6,  # frequency span for chirp signal, if enabled
        "sampling_divider": 3,
    },
    "timings_sec": {  # all in seconds
        "mixed_initilise": 600e-6,
        "read": 100e-6,
        "settle": 3e-3,
        "trigger": 8e-3,  # metronome of the internal trigger
        "buffer": 10e-6,
        "mw_pulse": 10e-6,
        "sampling_divider": 6,  # playback waveform resolution 2^X
    },
    "powers": {  # in dB presets from UI
        "acq_in": 0,
        "acq_out": -10,
        "drive": 0,  # only 0 is supported
        "mw_drive": 0,  # conncted to to PSG
        "psg_drive": -20,  # just a guess for now, output of psg
    },
    "averaging": {
        "seqc_averages": 1000,  # max 2**19 points
        "num_freqs": 10,
    },
    "gates": getvoltages(mdac),
    "temp": f"Mixing chamber {fridge.temp()} K",
}


# Dynamic variables
# create arrays of our sweeping paramters
freq_steps = np.linspace(params["mw"]["freqs"]["start"],
                      params["mw"]["freqs"]["stop"],
                      params["averaging"]["num_freqs"])

In [50]:
def sweep_esr_freq():
    """1D ESR frequency sweep"""
    monty.newrun("freq_sweep", params)

    # setup channels to match parameters
    esr.setupchannels(shfqc, params)
    esr.setupsequencers(shfqc, params, print_programs=False)
    esr.setup_command_tables(shfqc, params)
    time.sleep(1)  # wait to make sure everything processed in time

    # run the real experiment
    esr.synchchannels(shfqc, [shfqc.qa_channel_name] + shfqc.drive_channels)
    shfqc["measure"].spectroscopy.result.length(2)
    time.sleep(2)
    
    result = np.zeros((params["averaging"]["num_freqs"], 2), dtype=np.complex64)

    # upload command tables
    p1 = params["amplitude_volts"]["readout_pulse"]["P1"]
    p2 = params["amplitude_volts"]["readout_pulse"]["P2"]
    j = params["amplitude_volts"]["readout_pulse"]["J"]
    mw = params["mw"]["gain"]
    esr.movemeasurement(shfqc, p1, p2, j, mw, params)  # set amplitudes
    
    print("Sweeping P1, P2 and J")
    for i, mw in enumerate(tqdm(freq_steps, desc="Frequency")):
        esr.change_mw_freq(shfqc, mw, params)  # reupload sequence with new waveform
        esr.upload_command_tables(shfqc)
        result[i, :] = esr.run_esr_experiment(shfqc)

    results = {"data": result}
    monty.save(results)
    return results

In [52]:
sweep_esr_freq()

Started new run freq_sweep.31
Sweeping P1, P2 and J


Frequency:   0%|          | 0/10 [00:00<?, ?it/s]

Internal trigger:   0%|          | 0/100 [00:00<?, ?it/s]

Internal trigger:   0%|          | 0/100 [00:00<?, ?it/s]

Internal trigger:   0%|          | 0/100 [00:00<?, ?it/s]

...

Internal trigger:   0%|          | 0/100 [00:00<?, ?it/s]

Internal trigger:   0%|          | 0/100 [00:00<?, ?it/s]

Internal trigger:   0%|          | 0/100 [00:00<?, ?it/s]

Run finished and took 0:01:34.860473.
Saving to freq_sweep.31.xz
Saving to experiment.yaml


{'data': array([[-0.009601  +0.01012657j, -0.00963359+0.01013701j],
        [-0.00960018+0.0101265j , -0.00962901+0.01014185j],
        [-0.00959929+0.01012543j, -0.00962788+0.01013973j],
        [-0.00960155+0.01012763j, -0.00963589+0.01014498j],
        [-0.00960171+0.01012776j, -0.00962895+0.01014133j],
        [-0.00960026+0.01012623j, -0.00962884+0.01014053j],
        [-0.00959993+0.01012655j, -0.00963249+0.01014407j],
        [-0.00960146+0.01012695j, -0.00963569+0.01013851j],
        [-0.00960268+0.01012618j, -0.00963589+0.01014472j],
        [-0.00960109+0.0101267j , -0.00963557+0.01013761j]],
       dtype=complex64)}